# Important: Add competition dataset as input dataset first for this notebook to work

In [1]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit langchain
!pip install flash-attn --no-build-isolation
!pip install wtpsplit==2.1.1
!pip install syntok==1.4.4
!pip install omegaconf
!pip install wandb
!pip install --upgrade transformers trl
!pip install pandas numpy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 110.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapters 1.0.1 requires transformers~=4.45.2, but you have transformers 4.46.3 which is incompatible.


In [2]:
!git clone https://github.com/Reennon/gen-ai-nlp-lab-1

Cloning into 'gen-ai-nlp-lab-1'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 65 (delta 27), reused 41 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 236.33 KiB | 1.27 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [4]:
%cd gen-ai-nlp-lab-1
!ls

/content/gen-ai-nlp-lab-1
notebooks  params  poetry.lock	pyproject.toml	README.md  src


In [6]:
import os
import torch
import pandas as pd

from langchain_core.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from transformers import PreTrainedTokenizerBase, BitsAndBytesConfig
from tqdm import tqdm
from torch.utils.data import Dataset
from datasets import Dataset
from src.prompts.prompts import (NERPrompt1, NERPrompt2, NERPrompt3, NERPrompt4)
from src.prompts.examples import (
    FIFTEEN_SHOT_EXAMPLES_DICT,
    ELEVEN_SHOT_EXAMPLES_DICT,
    FIVE_SHOT_EXAMPLES_DICT,
    THREE_SHOT_EXAMPLES_DICT,
    ZERO_SHOT_EXAMPLES_DICT
)
from omegaconf import OmegaConf
from google.colab import userdata

In [7]:
QUANTIZE_4BIT = False
# device   = "cuda:0"
device = "cuda:0"

In [8]:
parameters = OmegaConf.load("./params/aya_23_8b.yml")

In [9]:
login(userdata.get('hf_key'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
checkpoint = "CohereForAI/aya-23-8b"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
seq_length = parameters.baseline.max_new_tokens
tokenizer.model_max_length = seq_length
max_seq_length = seq_length
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map="auto",  # Automatically map to GPUs
    attn_implementation="flash_attention_2",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [12]:
silver_train_df = pd.read_csv("./data/silver_train.csv")
silver_test_df = pd.read_csv("./data/silver_test.csv")

In [13]:
from src.prompts.prompts import BasePrompt

class NERPrompt1(BasePrompt):
    template: str = """Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей ("label"):
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей.
Та не змінюй відмінків знайдених сутностей.
Виводь сутності з вхідного тексту, а не прикладів.
Перевір, чи справді знайдена сутність відповідає категорії сутностей.
Якщо знайдена сутність не відповідає її категорії, то її не слід включати у відповідь.

Нижче наведено приклади формату та стилю розпізнавання сутностей:
{examples}

ВХІДНИЙ ТЕКСТ:
{text}

ЗНАЙДЕНІ СУТНОСТІ:"""
    input_variables: list[str] = ["text", "examples"]

In [14]:
example_template: str = """
ПРИКЛАД ТЕКСТУ:
{example_text}
ЗНАЙДЕНІ СУТНОСТІ З ТЕКСТУ:
{example_labels}
"""

def construct_prompt(
    prompt_template: PromptTemplate,
    few_shot_dict: dict[str, str],
    text: str
) -> str:
    examples = "".join([
            example_template.format(
                example_text=example["text"],
                example_labels=example["labels"]
            ) for example in few_shot_dict
        ])
    prompt = prompt_template.format(
        examples=examples,
        text=text,
    )

    return prompt

prompt = construct_prompt(
    prompt_template=NERPrompt1().prompt_template,
    few_shot_dict=ELEVEN_SHOT_EXAMPLES_DICT,
    text=""
)
print(prompt)

Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей ("label"):
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей. 
Та не змінюй відмінків знайдених сутностей.
Виводь сутності з вхідного тексту, а не при

In [15]:
tokenized_prompt = tokenizer.encode(prompt)
decoded_prompt = tokenizer.decode(tokenized_prompt)
prompt_len = len(tokenizer.tokenize(prompt))
print(decoded_prompt), prompt_len

Token indices sequence length is longer than the specified maximum sequence length for this model (2938 > 800). Running this sequence through the model will result in indexing errors


<BOS_TOKEN>Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей ("label"):
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей. 
Та не змінюй відмінків знайдених сутностей.
Виводь сутності з вхідного текст

(None, 2937)

In [16]:
def get_message_format(prompts):
  messages = []

  for p in prompts:
    messages.append(
        [{"role": "user", "content": p}]
      )

  return messages

In [79]:
import ast
import json
import logging
import re
from typing import Union


def sanitize_json(raw_json: str) -> str:
    """
    Cleans raw JSON by removing characters that could break the structure.
    For example, removes unmatched closing parentheses.
    """
    # Remove extra unmatched closing parentheses
    stack = []
    clean_content = []

    for char in raw_json:
        if char == '(':
            stack.append(char)
        elif char == ')':
            if stack:
                stack.pop()
            else:
                continue  # Skip unmatched closing parentheses
        clean_content.append(char)

    # Remove any dangling commas before a closing brace or bracket
    clean_json = ''.join(clean_content)
    clean_json = re.sub(r',\s*([\}\]])', r'\1', clean_json)

    return clean_json


def try_to_extract_dict_from_json(
    raw_output: str,
) -> Union[list[dict[str, str]], str]:
    pattern = r'```json\s*\n(?P<json>([\s\S]*?))\n```'
    matches = re.finditer(pattern, raw_output, re.MULTILINE)

    dict_output = ""
    for match in matches:
        json_content = match.group('json')
        try:
            dict_output = json.loads(json_content)

            return dict_output
        except json.JSONDecodeError as e:
            logging.debug("Invalid JSON:", e)
        except ValueError as e:
            logging.debug("ValueError:", e)

            return raw_output
    else:
        try:
            dict_output = ast.literal_eval(raw_output.strip("```json"))
        except SyntaxError as e:
            logging.debug(f"Got Syntax Error: {e} for output: {raw_output[:30]}; Sanitizing...")
            raw_output = sanitize_json(raw_output)
            try:
                dict_output = ast.literal_eval(raw_output.strip("```json"))
            except SyntaxError as e:
                logging.debug(f"Got another Syntax Error: {e} for output: {raw_output[:30]}; Returning raw output.")
                return raw_output
        except ValueError as e:
            logging.debug(f"Got Value Error: {e} for output: {raw_output[:30]}; Returning raw output.")

            return raw_output

    return dict_output

In [ ]:
out_labels = {}
max_seq_length = prompt_len + seq_length

split_from, split_to = 0, 43

for id in tqdm(silver_test_df.id.unique()[split_from: split_to], desc="Text progress"):
    inputs: list[str] = silver_test_df.loc[silver_test_df.loc[:, "id"] == id, "text"].to_list()
    inputs: list[str] = [construct_prompt(
        prompt_template=NERPrompt1().prompt_template,
        few_shot_dict=THREE_SHOT_EXAMPLES_DICT,
        text=input
    ) for input in inputs]
    print("inputs len")
    print([len(tokenizer.encode(input)) for input in inputs])
    print()
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    input_ids = input_ids.to(model.device)
    #input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])
    # Generate corrections
    # Check if the model is wrapped in DataParallel
    gen_tokens = model.generate(
            input_ids,
            temperature=parameters.baseline.temperature,
            top_p=parameters.baseline.top_p,
            top_k=parameters.baseline.top_k,
            max_new_tokens=seq_length,
            do_sample=True,
    )

    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[dict] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )

    print("output len")
    print(len(tokenizer.encode(str(outputs))))
    print("output")
    print(id, outputs)

    out_labels[id] = outputs

print(f"All texts extracted from {split_from} to {split_to}")

Text progress:   0%|          | 0/43 [00:00<?, ?it/s]

inputs len
[3427, 3232]



Text progress:   2%|▏         | 1/43 [01:31<1:04:00, 91.44s/it]

output len
1097
output
e29896ab781b5dbb97ae3f3f7862fa681e9d70a5e63866024e2473b317a25637 ["[{'label': 'LOG', 'text': 'департаменту культури і туризму Кіровоградської ОДА'}, {'label': 'LOC', 'text': 'село Моринці'}, {'label': 'LOC', 'text': 'Черкаської області'}, {'label': 'LOC', 'text': 'село Кирилівка'}, {'label': 'LOC', 'text': 'Звенигородського району'}, {'label': 'LOG', 'text': 'музейний комплекс'}, {'label': 'LOG', 'text': 'музей'}, {'label': 'LOG', 'text': 'хатина'}, {'label': 'LOG', 'text': 'садиба'}, {'label': 'LOG', 'text': 'хата'}, {'label': 'LOG', 'text': 'місцевий музей'}, {'label': 'LOG', 'text': 'меморіальний музей'}, {'label': 'LOG', 'text': 'зали'}, {'label': 'LOG', 'text': 'експонати'}, {'label': 'LOG', 'text': 'меблі'}, {'label': 'LOG', 'text': 'хата родини Шевченка'}, {'label': 'LOG', 'text': 'мійєтку пана Енгельгарта'}, {'label': 'LOG', 'text': 'село'}, {'label': 'LOG', 'text': 'село'}, {'label': 'LOG', 'text': 'село'}, {'label': 'LOG', 'text': 'село'}, {'label': 'LO

In [ ]:
import ast
import json
import re

def clean_incomplete_entries(data: str) -> str:
    pattern = r"\{[^}]*\}"  # Matches everything enclosed in {}

    # Find all matches of complete entries
    complete_entries = re.findall(pattern, data)

    # Join back into a single cleaned string
    cleaned_data = ', '.join(complete_entries)

    return cleaned_data

def filter_unique_dicts(dict_list):
    seen = set()
    unique_dicts = []

    for d in dict_list:
        # Convert dictionary to a frozenset of its items (immutable and hashable)
        items = frozenset(d.items())
        if items not in seen:
            seen.add(items)
            unique_dicts.append(d)

    return unique_dicts

def join_split_dicts(entities):
  some_list = []
  for entity_list in entities:
    some_list.extend(entity_list)

  return some_list

def postprocess(entities):
  result_dict_list = []
  for e in entities:
    e = clean_incomplete_entries(e)
    e = ast.literal_eval(e)
    e = list(e)
    e = filter_unique_dicts(e)
    result_dict_list.extend(e)

  result_dict_list = filter_unique_dicts(result_dict_list)

  return result_dict_list

def filter_dicts_by_text(dict_list, original_text):
    return [d for d in dict_list if 'text' in d and d['text'] in original_text]

silver_test_predictions_df = pd.DataFrame.from_dict([out_labels]).T
silver_test_predictions_df.columns = ["entities"]
silver_test_predictions_df.index.name = "id"
silver_test_predictions_df = silver_test_predictions_df.reset_index()
silver_test_predictions_df.loc[:, "entities"] = \
  silver_test_predictions_df.loc[:, "entities"].apply(lambda e: postprocess(e))
silver_test_df_concatenated = (
    silver_test_df.groupby("id", as_index=False)["text"].apply(" ".join).reset_index()
)
silver_test_predictions_df = pd.merge(
    silver_test_predictions_df,
    silver_test_df_concatenated.loc[:, ["text", "id"]],
    how="left",
    on="id"
)
silver_test_predictions_df.loc[:, "entities"] = \
  silver_test_predictions_df.apply(lambda e: filter_dicts_by_text(e["entities"], e["text"]), axis=1)
silver_test_predictions_df.loc[:, "entities_dumps"] = \
  silver_test_predictions_df.loc[:, "entities"].apply(lambda e: json.dumps(e))
silver_test_predictions_df


In [ ]:
silver_test_predictions_df.to_csv(f"./data/submission_{split_from}_{split_to}.csv", index=False)